# 自动求导
## 什么是自动求导
自动求导(AUTOMATIC DIFFERENTIATION 简称 AD) 或许是目前你从未听过且最有用的计算技术之一。如果你的工作涉及到实数计算，那么理解 AD 或许将会对你的工作有所帮助。  
自动求导，也被称为算法求导(algorithmic differentiation)或计算求导(computational differentiation)，是一组使用数值方法对某个函数，通过编程的方式进行求导的技术。可以求任意阶的导数，求导过程是自动的，并且能够保证足够的精度，以及较小的时间复杂度。

自动求导的两种模式：
* 正向积累
* 反向积累、反向传递

假设我们想对函数y=2x^Tx关于列向量求导。

In [2]:
import torch

x=torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

在计算y关于x的梯度之前，我们需要一个地方来存储梯度。（梯度的本意是一个向量，表示某函数在该点处的方向导数沿该方向取得最大值，即函数在该点处沿着该方向（此梯度的方向）变化最快，变化率最大（为该梯度的模）。）

In [4]:
x.requires_grad_(True)
x.grad #x的梯度默认值是None

计算y

In [5]:
y=2*torch.dot(x,x)
y

tensor(28., grad_fn=<MulBackward0>)

通过调用反向传播函数来自动计算y关于x每个分量的梯度

In [6]:
y.backward() #这里反向转播就是自动求导y=2*x*x求导后为4*x
x.grad

tensor([ 0.,  4.,  8., 12.])

In [7]:
#所以这里可以验证一下
x.grad==4*x

tensor([True, True, True, True])

计算x的另一个函数

In [9]:
#在默认情况下，Pytorch会累积梯度，我们需要清除之前的值
x.grad.zero_()
y=x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

深度学习中，我们的目的不是计算微分矩阵，而是批量中每个样本单独计算的偏导数之和。

In [14]:
x.grad.zero_()
y=x*x
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

将某些计算移到记录的计算图之外

In [15]:
x.grad.zero_()
y=x*x
u=y.detach() #这里相当于对y进行不对x进行求导，相当于把u变成了一个常数
z=u*x
z.sum().backward()
x.grad==u

tensor([True, True, True, True])